In [1]:
import pandas as pd
from sqlalchemy import create_engine
import datetime 

In [2]:
IN_file = "IN_COVID.csv"
IN_df = pd.read_csv(IN_file)
IN_df.head()

,Date,New Cases,7-Day Moving Avg
0,Dec 1 2020,5396,5405
1,Nov 30 2020,5665,5437
2,Nov 29 2020,4304,5422
3,Nov 28 2020,4471,5689
4,Nov 27 2020,5643,6032


- I chose this dataset from the CDC by selecting Indiana and downloading the csv file that shows new cases compared to 7 day moving average by date. The reason we selected this dataset was to compare the data to our Google mobility dataset. We hope to compare cases and where people were going to see if there is a correlation or a causation to the rise in cases.
- Check date format

In [3]:
IN_df.dtypes

Date                object
New Cases            int64
7-Day Moving Avg     int64
dtype: object

- Needed to get data types to see what the date column was so I can convert the format

In [4]:
IN_df['Date'] = pd.to_datetime(IN_df['Date'], format='%Y%m%d', infer_datetime_format=True)

In [5]:
IN_df.dtypes

Date                datetime64[ns]
New Cases                    int64
7-Day Moving Avg             int64
dtype: object

In [6]:
# renamed columns
IN_df = IN_df.rename(columns={'New Cases': 'New_Cases',
                      '7-Day Moving Avg': 'Seven_Day_Moving_Avg'
                     }
                      )

In [7]:
# Set dataframe index to the date
IN_df.set_index('Date')

,New_Cases,Seven_Day_Moving_Avg
Date,,
2020-12-01,5396,5405
2020-11-30,5665,5437
2020-11-29,4304,5422
2020-11-28,4471,5689
2020-11-27,5643,6032
...,...,...
2020-01-26,0,0
2020-01-25,0,0
2020-01-24,0,0


In [8]:
# Sort dataframe by date - display the last 30 days reported
IN_df.sort_index(ascending=False)
IN_df.set_index('Date')

,New_Cases,Seven_Day_Moving_Avg
Date,,
2020-12-01,5396,5405
2020-11-30,5665,5437
2020-11-29,4304,5422
2020-11-28,4471,5689
2020-11-27,5643,6032
...,...,...
2020-01-26,0,0
2020-01-25,0,0
2020-01-24,0,0


In [9]:
conn = 'postgres:postgres@localhost:5432/covid-indiana'
engine = create_engine(f'postgresql://{conn}')

In [10]:
engine.table_names()

['mobility_per_date', 'in_cases']

In [11]:
IN_df.to_sql(name='indiana_newcases', con=engine, if_exists='append', index=True)